In [1]:
import sys
from pathlib import Path

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

IN_DIR = DIR/'examples'/'inputs'
OUT_DIR = DIR/'examples'/'outputs'

%load_ext autoreload
%autoreload 2

high_definition = False


In [ ]:
# Download elevation data

bounds = [174, -37, 176, -36]
names = wt.get_srtm_tile_names(bounds)
print('Tiles to download =', names)

username = None
password = None
if high_definition:
    path = IN_DIR/'elevations-hd'
    try:
        username = wt.get_secret('NASA_USERNAME')
        password = wt.get_secret('NASA_PASSWORD')
    except:
        pass
else:
    path = IN_DIR/'elevations-sd'
    
wt.download_elevation_data_nasa(bounds, path, high_definition=high_definition,
  username=username, password=password)

In [ ]:
# Create SPLAT transmitters data

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
wt.create_splat_transmitter_data(in_path, out_path)
% ls {out_path}

In [ ]:
# Create SPLAT elevation data

if high_definition:
    in_path = IN_DIR/'elevations-hd'
else:
    in_path = IN_DIR/'elevations-sd'
out_path = OUT_DIR/'splat_files'

wt.create_splat_elevation_data(in_path, out_path, high_definition=high_definition)
% ls {out_path}

In [9]:
# Create coverage reports

transmitter_names = ['MyNetwork_11']
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)

% ls {out_path}

Timing create_coverage_reports...
2016-08-09 16:35:57.456815   Began process
2016-08-09 16:38:30.812679   Finished in 2.55 min
MyNetwork_11-ck.png  MyNetwork_11.png              MyNetwork_11.tif
MyNetwork_11-ck.ppm  MyNetwork_11.ppm
MyNetwork_11.kml     MyNetwork_11-site_report.txt


In [17]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path, delete_ppm=False)

% ls {path}

gdal_translate -of Gtiff -a_ullr 173.0 -35.00083 177.0 -38.0 -a_srs EPSG:4326 MyNetwork_11.png MyNetwork_11.tif
MyNetwork_11-ck.png  MyNetwork_11.png              MyNetwork_11.tif
MyNetwork_11-ck.ppm  MyNetwork_11.ppm
MyNetwork_11.kml     MyNetwork_11-site_report.txt


In [18]:
import re
import subprocess


re.search(r"<west>([0-9-][0-9\.]*)<\/west>", '').group(1)

AttributeError: 'NoneType' object has no attribute 'group'